# Introduction
[PyTorch] is one of the two most popular Deep Learning frameworks in Python, besides TensorFlow. Here is some key points when comparing the two:
- In terms of low or high level, PyTorch falls somewhere in between TensorFlow and Keras. No fit-and-predict interface, must be done by hand.
- PyTorch is prefered by research community with more customizations, as we normally see newly published architectures written in PyTorch.
- TensorFlow/Keras is better for production due to high-level interface and large deployment ecosystem.

[PyTorch]: https://github.com/pytorch/pytorch

# 1. Data manipulation

## 1.1. Computation

In [14]:
import numpy as np
import pandas as pd
import janitor
import torch

### Tensor
In PyTorch, we work most of the time with <code style='font-size:13px'><a href=https://pytorch.org/docs/stable/tensors.html>Tensor</a></code> whose operations are very much like NumPy's array. Being very natural to PyTorch, tensor operations are provided directly in the [mother package].

[mother package]: https://pytorch.org/docs/stable/torch.html

In [7]:
a = torch.tensor([
    [1., 2., 3.],
    [4., 5., 6.]
])

In [8]:
torch.rand_like(a)

tensor([[0.3158, 0.5845, 0.8687],
        [0.7701, 0.0564, 0.2115]])

### Autograd

In [13]:
x = torch.tensor([1,2,3]).reshape(-1,1)
x

tensor([[1],
        [2],
        [3]])

In [27]:
df = pd.read_csv('data/boston.csv')
df.head()

,crime_rate,land_rate,indus,chas,nox,room,age,distance,radial,tax,ptratio,black,lstat,price
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [40]:
x1 = torch.tensor(df.room)
x2 = torch.tensor(df.lstat)
yTrue = torch.tensor(df.price)

In [36]:
w1 = torch.rand(1, requires_grad=True)
w2 = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

In [43]:
yPred = w1*x1 + w2*x2 + b
loss = torch.nn.functional.mse_loss(yPred, yTrue)

In [44]:
loss.backward()

In [45]:
w1.grad

tensor([-143.3771])

In [46]:
w2.grad

tensor([-111.1677])

In [49]:
params = (w1, w2, b)
optimizer = torch.optim.SGD(params, lr=0.1)

# 2. Neural networks

PyTorch has two APIs for creating layers,
<code style='font-size:13px'><a href=https://pytorch.org/docs/stable/nn.html>nn</a></code>
(abbreviated for *neural network*) and
<code style='font-size:13px'><a href=https://pytorch.org/docs/stable/nn.functional.html>nn.functional</a></code>.
The first module provides object interface (that supports auto differentiation) and the second module provides function interface (easier to use). So, the best practice is using object interface for layers with trainable parameters such as recurrent or convolutional, and using function interface for loss functions or activation functions.

PyTorch has two APIs for creating models, where the recommended one is
<code style='font-size:13px'><a href=https://pytorch.org/docs/stable/generated/torch.nn.Module.html>nn.Module</a></code>,
being equivalent to functional API in Keras. To create a model, we inherit this class, define building blocks inside the <code style='font-size:13px'>\_\_init__()</code> method and design the neural network architecture with the <code style='font-size:13px'>foward()</code> method. We don't need to to specify the backward pass, as the submodule
<code style='font-size:13px'><a href=https://pytorch.org/docs/stable/autograd.html>autograd</a></code>
will handle it for us. The second API,
<code style='font-size:13px'><a href=https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>nn.Sequential</a></code>,
is good for simple architectures as well as small blocks of large networks, inception block of GooLeNet for example.

In [1]:
import torch
from torch import nn
from torch.autograd import Variable

In [2]:
torch.cuda.is_available()

False

In [56]:
x = torch.autograd.Variable(torch.Tensor([[1,2,3,4,5]]), requires_grad=True)
y = torch.sum(x**2)
y.backward() 
x.grad

tensor([[ 2.,  4.,  6.,  8., 10.]])

In [57]:
class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.

        D_in: input dimension
        H: dimension of hidden layer
        D_out: output dimension
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = nn.Linear(D_in, H) 
        self.linear2 = nn.Linear(H, D_out)

def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must 
        return a Variable of output data. We can use Modules defined in the 
        constructor as well as arbitrary operators on Variables.
        """
        h_relu = nn.functional.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred

In [72]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)
        
class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length

# References
- *pytorch.org - [Deep Learning with PyTorch: A 60-minute blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)*
- *towardsdatascience.com - [Understanding PyTorch with an example: a step-by-step tutorial](https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e)*
- *towardsdatascience.com - [PyTorch vs TensorFlow - spotting the difference](https://towardsdatascience.com/pytorch-vs-tensorflow-spotting-the-difference-25c75777377b)*
- *blog.paperspace.com - [PyTorch 101 advanced](https://blog.paperspace.com/pytorch-101-advanced/)*
- *poloclub.github.io - [CNN explainer](https://poloclub.github.io/cnn-explainer/)*
- https://cs230.stanford.edu/blog/pytorch/

In [ ]:
conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [ ]:
!pip3 install torch torchvision torchaudio --user

---
*&#9829; By Quang Hung x Thuy Linh &#9829;*